https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

In [ ]:
# !pip install pandas --user
# !pip install gensim --user
# !pip install scikit-learn --user
# pip3 install -U scikit-learn scipy matplotlib
# !pip install scikit-learn --user
# !pip install gensim==3.4.0
# !pip install smart_open==1.9.0
# !pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import smart_open
from gensim.models.keyedvectors import KeyedVectors

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from sklearn.metrics import classification_report

In [ ]:
# path = 'GoogleNews-vectors-negative300.bin.gz'
# model = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
# Cargar el dataset
df = pd.read_csv('career_prediction_dataset.csv')
print(df)

In [ ]:
categories = {
    "Healthcare and Medicine": [
        "Nurse", "Physician", "Pharmacist", "Physical Therapist", 
        "Dental Hygienist", "Pediatric Nurse", "Chiropractor", 
        "Radiologic Technologist", "Speech Therapist", "Occupational Therapist", 
        "Rehabilitation Counselor", "Pediatrician", "Genetic Counselor", 
        "Forensic Psychologist", "Public Health Analyst"
    ],
    "Engineering and Technology": [
        "Software Developer", "IT Support Specialist", "Biomedical Engineer", 
        "Data Analyst", "Industrial Engineer", "Mechanical Engineer", 
        "Environmental Engineer", "Electrical Engineer", "Aerospace Engineer", 
        "Mechanical Designer", "Robotics Engineer", "Civil Engineer", 
        "Electronics Design Engineer", "Web Developer", "Quality Control Inspector", 
        "Video Game Tester", "Game Developer", "IT Project Manager", "Database Analyst", "Software Quality Assurance Tester", "Database Administrator"
    ],
    "Science and Research": [
        "Research Scientist", "Biologist", "Astronomer", "Environmental Scientist", 
        "Biotechnologist", "Zoologist", "Forensic Scientist", "Geologist", 
        "Wildlife Biologist", "Marine Biologist", "Biomedical Researcher"
    ],
    "Business and Finance": [
        "Accountant", "Financial Analyst", "Financial Planner", "Financial Auditor", 
        "Financial Advisor", "Investment Banker", "Tax Accountant", "Insurance Underwriter", "Product Manager"
    ],
    "Marketing and Sales": [
        "Salesperson", "Marketing Manager", "Marketing Coordinator", "Event Planner", 
        "Real Estate Agent", "Market Research Analyst", "Marketing Copywriter", 
        "Social Media Manager", "Advertising Executive", "Public Relations Specialist", 
        "Market Researcher", "Event Photographer", "Chef", "Marketing Analyst"
    ],
    "Human Resources and Administration": [
        "Human Resources Manager", "HR Recruiter", "Administrative Officer", 
        "IT Project Manager"
    ],
    "Design and Creative Arts": [
        "Graphic Designer", "Architect", "Artist", "Fashion Designer", 
        "Musician", "Interior Designer", "Film Director", "Fashion Stylist", "Technical Writer"
    ],
    "Education and Counseling": [
        "Teacher", "Psychologist", "Elementary School Teacher", "Social Worker", 
        "Marriage Counselor", "Speech Pathologist", "Sports Coach", 
        "Rehabilitation Counselor", "Career Counselor"
    ],
    "Law and Public Safety": [
        "Lawyer", "Police Detective", "Police Officer", "Human Rights Lawyer", 
        "Customs and Border Protection Officer", "Diplomat", "Foreign Service Officer", 
        "Tax Collector", "Air Traffic Controller", "Airline Pilot", "Journalist"
    ],
    "Urban Planning and Environment": [
        "Urban Planner", "Construction Engineer", "Environmental Scientist", 
        "Environmental Engineer", "Forestry Technician", "Wildlife Conservationist"
    ]
}


inverse_categories = {v: k for k, values in categories.items() for v in values}

df['Field'] = df['Career'].apply(lambda x: inverse_categories[x] if x in inverse_categories else 'Other Specialized Fields')

In [ ]:
# Codificación de la columna 'Career'
df['Career_encoded'] = LabelEncoder().fit_transform(df['Career'])
df['Field_encoded'] = LabelEncoder().fit_transform(df['Field'])

In [ ]:
field_counts = df['Field'].value_counts()
sub_df = df[df['Field'].isin(field_counts[field_counts >= 7].index)]
sub_df.corr()

In [ ]:
# X = df.drop(['Career', 'Field', 'Career_encoded', 'Field_encoded'], axis=1)
# Y = df['Field_encoded']

# train_X = X[:int(0.8*len(X))]
# train_Y = Y[:int(0.8*len(Y))]
# test_X = X[int(0.8*len(X)):]
# test_Y = Y[int(0.8*len(Y)):]

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['Career', 'Field', 'Career_encoded', 'Field_encoded'], axis=1)
Y = df['Field_encoded']

poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

# Supongamos que X y Y son tus datos y etiquetas
# Primero dividimos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Ahora dividimos el conjunto de entrenamiento en entrenamiento y validación (80% entrenamiento, 20% validación)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

model = RandomForestClassifier()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

print(classification_report(Y_test, Y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=100,  # Number of parameter settings sampled
    cv=5,  # Number of folds in cross-validation
    verbose=2,  # Controls the verbosity
    random_state=42,  # For reproducibility
    n_jobs=-1  # Number of jobs to run in parallel
)

# Fit RandomizedSearchCV to your data
# Replace X_train and y_train with your training data
random_search.fit(X_train, Y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict on test data
# Replace X_test with your test data
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Best parameters: {random_search.best_params_}')
print(f'Accuracy: {accuracy}')


In [ ]:
# how to save my sklearn model?
import joblib
joblib.dump(model, 'model.pkl')

In [ ]:
importances = model.feature_importances_

# feature_names = [X.columns[i] for i in range(X.shape[1])]
feature_names = [str(i) for i in range(X.shape[1])]

forest_importances = pd.Series(importances, index=feature_names)
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
fig, ax = plt.subplots(1,1, figsize=(10, 6))
forest_importances.plot.bar(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
# model.get_params()